In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 28s - loss: 0.4627 - auc: 0.9066 - accuracy: 0.8224 - val_loss: 0.4339 - val_auc: 0.9893 - val_accuracy: 0.8047
Epoch 2/30
40/40 - 27s - loss: 0.1102 - auc: 0.9924 - accuracy: 0.9606 - val_loss: 0.0387 - val_auc: 0.9992 - val_accuracy: 0.9948
Epoch 3/30
40/40 - 26s - loss: 0.0404 - auc: 0.9998 - accuracy: 0.9917 - val_loss: 0.0281 - val_auc: 1.0000 - val_accuracy: 0.9948
Epoch 4/30
40/40 - 26s - loss: 0.0243 - auc: 1.0000 - accuracy: 0.9968 - val_loss: 0.0237 - val_auc: 1.0000 - val_accuracy: 0.9948
Epoch 5/30
40/40 - 27s - loss: 0.0146 - auc: 1.0000 - accuracy: 0.9992 - val_loss: 0.0118 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 6/30
40/40 - 26s - loss: 0.0130 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 0.0117 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 7/30
40/40 - 27s - loss: 0.0097 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 0.0104 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 8/30
40/40 - 27s - loss: 0.0080 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

318
68


In [8]:
prediction

array([[9.9892670e-01],
       [9.9978489e-01],
       [9.7925401e-01],
       [9.4410330e-01],
       [7.6400512e-03],
       [9.9999666e-01],
       [9.9925154e-01],
       [9.9999905e-01],
       [9.9995136e-01],
       [9.9884510e-01],
       [1.0000000e+00],
       [9.9956685e-01],
       [9.9999988e-01],
       [9.9067724e-01],
       [9.9730468e-01],
       [1.0000000e+00],
       [9.9457777e-01],
       [1.0000000e+00],
       [7.1204197e-01],
       [4.8667868e-03],
       [9.9676681e-01],
       [1.0000000e+00],
       [9.9923515e-01],
       [9.9977392e-01],
       [9.5382047e-01],
       [9.9945241e-01],
       [9.9960107e-01],
       [7.3467910e-02],
       [9.9949992e-01],
       [9.9995708e-01],
       [9.7976607e-01],
       [9.5390761e-01],
       [2.7210973e-04],
       [9.9952996e-01],
       [9.6597189e-01],
       [9.9881172e-01],
       [9.5022458e-01],
       [9.9902892e-01],
       [9.9605846e-01],
       [9.9603939e-01],
       [9.8403072e-01],
       [9.998182

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 425ms/step - loss: 0.0282 - auc: 1.0000 - accuracy: 0.9948


[0.02820645272731781, 1.0, 0.9948186278343201]